# Vizzu 可视化图表组件

`Vizzu`组件在Panel中渲染[Vizzu](https://lib.vizzuhq.com/)图表。注意，要在notebook中使用`Vizzu`组件，必须在加载Panel扩展时将'vizzu'作为参数传递，以确保初始化vizzu.js。

底层实现为`panel.pane.Vizzu`，参数基本一致，参考文档：https://panel.holoviz.org/reference/panes/Vizzu.html


In [1]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

`PnVizzu`组件可以根据`config`定义如何绘制数据（以列字典或DataFrame的形式定义）：


In [2]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnVizzu :object="data" 
           :config="{'geometry': 'rectangle', 'x': 'Name', 
                     'y': 'Weight', 'title': 'Weight by person'}"
           :duration="400" 
           :height="400" 
           sizing_mode="stretch_width" 
           :tooltip="True" />
</template>

<script lang='py'>
import numpy as np

# Create sample data
data = {
    'Name': ['Alice', 'Bob', 'Ted', 'Patrick', 'Jason', 'Teresa', 'John'],
    'Weight': 50+np.random.randint(0, 10, 7)*10
}
</script>

{"vue": "<!-- --plugins vpanel --show-code -->\n<template>\n  <PnVizzu :object=\"data\" \n           :config=\"{'geometry': 'rectangle', 'x': 'Name', \n                     'y': 'Weight', 'title': 'Weight by person'}\"\n           :duration=\"400\" \n           :height=\"400\" \n           sizing_mode=\"stretch_width\" \n           :tooltip=\"True\" />\n</template>\n\n<script lang='py'>\nimport numpy as np\n\n# Create sample data\ndata = {\n    'Name': ['Alice', 'Bob', 'Ted', 'Patrick', 'Jason', 'Teresa', 'John'],\n    'Weight': 50+np.random.randint(0, 10, 7)*10\n}\n</script>\n", "setup": ""}



Vizzu的主要卖点之一是在数据或`config`更新时的动态动画。例如，如果我们更改"geometry"，可以看到动画在两种状态之间平滑过渡。


In [3]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnVizzu ref="vizzu_ref" 
           :object="data" 
           :config="config.value"
           :duration="400" 
           :height="400" 
           sizing_mode="stretch_width" 
           :tooltip="True" />
  <PnRow>
    <PnButton @click="changeToCircle()">Change to Circle</PnButton>
    <PnButton @click="changeToArea()">Change to Area</PnButton>
    <PnButton @click="changeToRectangle()">Change to Rectangle</PnButton>
  </PnRow>
</template>

<script lang='py'>
import numpy as np
from vuepy import ref

# Create sample data
data = {
    'Name': ['Alice', 'Bob', 'Ted', 'Patrick', 'Jason', 'Teresa', 'John'],
    'Weight': 50+np.random.randint(0, 10, 7)*10
}

config = ref({'geometry': 'rectangle', 'x': 'Name', 'y': 'Weight', 
              'title': 'Weight by person'})
vizzu_ref = ref(None)

def changeToCircle():
    config.value = {**config.value, 'geometry': 'circle'}
    
def changeToArea():
    config.value = {**config.value, 'geometry': 'area'}
    
def changeToRectangle():
    vizzu = vizzu_ref.value.unwrap()
    vizzu.animate({'geometry': 'rectangle'})
    config.value = vizzu.config
</script>

{"vue": "<!-- --plugins vpanel --show-code -->\n<template>\n  <PnVizzu ref=\"vizzu_ref\" \n           :object=\"data\" \n           :config=\"config.value\"\n           :duration=\"400\" \n           :height=\"400\" \n           sizing_mode=\"stretch_width\" \n           :tooltip=\"True\" />\n  <PnRow>\n    <PnButton @click=\"changeToCircle()\">Change to Circle</PnButton>\n    <PnButton @click=\"changeToArea()\">Change to Area</PnButton>\n    <PnButton @click=\"changeToRectangle()\">Change to Rectangle</PnButton>\n  </PnRow>\n</template>\n\n<script lang='py'>\nimport numpy as np\nfrom vuepy import ref\n\n# Create sample data\ndata = {\n    'Name': ['Alice', 'Bob', 'Ted', 'Patrick', 'Jason', 'Teresa', 'John'],\n    'Weight': 50+np.random.randint(0, 10, 7)*10\n}\n\nconfig = ref({'geometry': 'rectangle', 'x': 'Name', 'y': 'Weight', \n              'title': 'Weight by person'})\nvizzu_ref = ref(None)\n\ndef changeToCircle():\n    config.value = {**config.value, 'geometry': 'circle'}\n    \


## 列类型

`PnVizzu`支持两种列类型：

- `'dimension'`：通常用于非数值数据和/或图表的独立维度（例如x轴）
- `'measure'`：数值通常用于图表的因变量（例如y轴值）

`PnVizzu`组件会根据数据的dtypes自动推断类型，但在某些情况下，可能需要使用`column_types`参数显式覆盖列的类型。一个常见的例子是在x轴上绘制整数时，通常会被视为"measure"，但在折线图或条形图的情况下应该被视为独立维度。

下面的示例演示了这种情况，这里我们希望将"index"视为独立变量，并使用`column_types={'index': 'dimension'}`覆盖默认推断的类型：


In [4]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnVizzu :object="df" 
           :column_types="{'index': 'dimension'}" 
           :config="{'x': 'index', 'y': 'Y', 'geometry': 'line'}"
           :height="300" 
           sizing_mode="stretch_width" />
</template>

<script lang='py'>
import numpy as np
import pandas as pd

# Create sample data
df = pd.DataFrame(np.random.randn(50), columns=list('Y')).cumsum()
</script>

{"vue": "<!-- --plugins vpanel --show-code -->\n<template>\n  <PnVizzu :object=\"df\" \n           :column_types=\"{'index': 'dimension'}\" \n           :config=\"{'x': 'index', 'y': 'Y', 'geometry': 'line'}\"\n           :height=\"300\" \n           sizing_mode=\"stretch_width\" />\n</template>\n\n<script lang='py'>\nimport numpy as np\nimport pandas as pd\n\n# Create sample data\ndf = pd.DataFrame(np.random.randn(50), columns=list('Y')).cumsum()\n</script>\n", "setup": ""}



## 预设

Vizzu提供了各种[预设图表类型](https://lib.vizzuhq.com/latest/examples/presets/)。在`PnVizzu`组件中，您可以通过在`config`中提供`'preset'`作为键来使用这些预设。在下面的示例中，我们动态创建一个`config`，根据`RadioButtonGroup`切换`preset`：


In [5]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnRow>
    <PnRadioButtonGroup 
        v-model="chart_type.value" 
        :options="{'Stream': 'stream', 'Bar': 'stackedColumn'}" 
        align="center" />
  </PnRow>
  <PnVizzu :object="agg"
           :config="getConfig()"
           :column_types="{'p_year': 'dimension'}"
           :height="500"
           sizing_mode="stretch_width"
           :style="{
             'plot': {
               'xAxis': {
                 'label': {
                   'angle': '-45deg'
                 }
               }
             }
           }" />
</template>

<script lang='py'>
import pandas as pd
from vuepy import ref

# Load data
windturbines = pd.read_csv('https://datasets.holoviz.org/windturbines/v1/windturbines.csv')
agg = windturbines.groupby(['p_year', 't_manu'])[['p_cap']]\
      .sum().sort_index(level=0).reset_index()

# Chart type selection
chart_type = ref('stream')

def getConfig():
    return {
        'preset': chart_type.value, 
        'x': 'p_year', 
        'y': 'p_cap', 
        'stackedBy': 't_manu'
    }
</script>

{"vue": "<!-- --plugins vpanel --show-code -->\n<template>\n  <PnRow>\n    <PnRadioButtonGroup \n        v-model=\"chart_type.value\" \n        :options=\"{'Stream': 'stream', 'Bar': 'stackedColumn'}\" \n        align=\"center\" />\n  </PnRow>\n  <PnVizzu :object=\"agg\"\n           :config=\"getConfig()\"\n           :column_types=\"{'p_year': 'dimension'}\"\n           :height=\"500\"\n           sizing_mode=\"stretch_width\"\n           :style=\"{\n             'plot': {\n               'xAxis': {\n                 'label': {\n                   'angle': '-45deg'\n                 }\n               }\n             }\n           }\" />\n</template>\n\n<script lang='py'>\nimport pandas as pd\nfrom vuepy import ref\n\n# Load data\nwindturbines = pd.read_csv('https://datasets.holoviz.org/windturbines/v1/windturbines.csv')\nagg = windturbines.groupby(['p_year', 't_manu'])[['p_cap']]\\\n      .sum().sort_index(level=0).reset_index()\n\n# Chart type selection\nchart_type = ref('stream')\n\


## 交互控制

`PnVizzu`组件公开了许多选项，可以从Python和JavaScript更改。尝试交互式地测试这些参数的效果：


In [6]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnVizzu ref="vizzu_ref" 
           :object="data" 
           :config="dict(config.value)"
           :duration="duration.value" 
           :height="400" 
           sizing_mode="stretch_width" 
           :tooltip="tooltip.value" />
  <PnRow>
    <PnCol>
      <PnCheckbox v-model="tooltip.value" name="Show Tooltip" />
      <PnIntSlider v-model="duration.value" 
                   name="Animation Duration" 
                   :start="100" 
                   :end="2000" 
                   :step="100" />
    </PnCol>
    <PnCol>
      <PnButton @click="changeToCircle()">Change to Circle</PnButton>
      <PnButton @click="changeToArea()">Change to Area</PnButton>
      <PnButton @click="changeToRectangle()">Change to Rectangle</PnButton>
    </PnCol>
  </PnRow>
</template>

<script lang='py'>
import numpy as np
from vuepy import ref

# Create sample data
data = {
    'Name': ['Alice', 'Bob', 'Ted', 'Patrick', 'Jason', 'Teresa', 'John'],
    'Weight': 50+np.random.randint(0, 10, 7)*10
}

# Control parameters
tooltip = ref(True)
duration = ref(400)
vizzu_ref = ref(None)
config = ref({
    'geometry': 'rectangle', 'x': 'Name', 'y': 
    'Weight', 'title': 'Weight by person'
})

def changeToCircle():
    config.value.geometry = 'circle'
    
def changeToArea():
    config.value.geometry = 'area'
    
def changeToRectangle():
    config.value.geometry = 'rectangle'
</script>

{"vue": "<!-- --plugins vpanel --show-code -->\n<template>\n  <PnVizzu ref=\"vizzu_ref\" \n           :object=\"data\" \n           :config=\"dict(config.value)\"\n           :duration=\"duration.value\" \n           :height=\"400\" \n           sizing_mode=\"stretch_width\" \n           :tooltip=\"tooltip.value\" />\n  <PnRow>\n    <PnCol>\n      <PnCheckbox v-model=\"tooltip.value\" name=\"Show Tooltip\" />\n      <PnIntSlider v-model=\"duration.value\" \n                   name=\"Animation Duration\" \n                   :start=\"100\" \n                   :end=\"2000\" \n                   :step=\"100\" />\n    </PnCol>\n    <PnCol>\n      <PnButton @click=\"changeToCircle()\">Change to Circle</PnButton>\n      <PnButton @click=\"changeToArea()\">Change to Area</PnButton>\n      <PnButton @click=\"changeToRectangle()\">Change to Rectangle</PnButton>\n    </PnCol>\n  </PnRow>\n</template>\n\n<script lang='py'>\nimport numpy as np\nfrom vuepy import ref\n\n# Create sample data\ndata 


## API

### 属性

| 属性名        | 说明                                     | 类型                | 默认值 |
| ------------- | ---------------------------------------- | ------------------- | ------ |
| value         | 以Python数组字典或DataFrame表示的数据     | ^[dict\|pd.DataFrame] | —      |
| animation     | 动画设置                                 | ^[dict]             | {}     |
| config        | 包含渲染特定静态图表或动画图表状态所需的所有参数 | ^[dict]        | {}     |
| columns       | 可选的列定义。如果未定义，将从数据中推断   | ^[list]             | None   |
| column_types  | 列类型定义，覆盖自动推断的类型            | ^[dict]             | {}     |
| tooltip       | 是否在图表上启用工具提示                  | ^[bool]             | False  |
| duration      | 动画持续时间（毫秒）                      | ^[int]              | 500    |
| style         | 图表样式配置                             | ^[dict]             | {}     |

### 方法

| 方法名    | 说明                            | 类型                         |
| --------- | ------------------------------- | ---------------------------- |
| animate   | 接受一个新的'data'、'config'和'style'值的字典，用于更新图表 | ^[Callable]`(obj: dict) -> None` |
| stream    | 向图表流式传输新数据            | ^[Callable]`(data: dict) -> None` |
| patch     | 修补数据中的一行或多行          | ^[Callable]`(data: dict) -> None` |
| controls  | 返回控制面板组件                | ^[Callable]`(jslink=bool) -> Panel` |


## Controls

In [8]:
##controls
import pandas as pd
import numpy as np
import panel as pn

pn.extension('vizzu')
data = {
    'Name': ['Alice', 'Bob', 'Ted', 'Patrick', 'Jason', 'Teresa', 'John'],
    'Weight': 50+np.random.randint(0, 10, 7)*10
}

vizzu = pn.pane.Vizzu(
    data, config={'geometry': 'rectangle', 'x': 'Name', 'y': 'Weight', 'title': 'Weight by person'},
    duration=400, height=400, sizing_mode='stretch_width', tooltip=True
)

pn.Row(vizzu.controls(), vizzu)

Row
    [0] Tabs
        [0] WidgetBox(margin=(5, 10), name='Controls')
            [0] StaticText(value='<b>Controls</b>')
            [1] DictInput(description='Animation settings (..., name='Animation', serializer='json', type=<class 'dict'>)
            [2] DictInput(description='The config contains a..., name='Config', serializer='json', type=<class 'dict'>, value={'geometry': 'rectangle', ...})
            [3] IntInput(description='The config contains a..., name='Duration', value=400)
            [4] DictInput(description='Style configuration o..., name='Style', serializer='json', type=<class 'dict'>)
            [5] Checkbox(name='Tooltip', value=True)
        [1] WidgetBox(margin=(5, 10), name='Layout')
            [0] StaticText(value='<b>Layout</b>')
            [1] TextInput(description='String identifier f..., name='Name', value='Vizzu00308')
            [2] LiteralInput(description='Whether the object should..., name='Align', serializer='json', value='start')
            [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio', serializer='json')
            [4] ListInput(description='CSS classes t..., name='Css classes', serializer='json', type=<class 'list'>)
            [5] IntInput(description='The height of the compone..., name='Height', start=0, value=400)
            [6] IntInput(description='Minimal width o..., name='Min width', start=0)
            [7] IntInput(description='Minimal height o..., name='Min height', start=0)
            [8] IntInput(description='Maximum width o..., name='Max width', start=0)
            [9] IntInput(description='Maximum height o..., name='Max height', start=0)
            [10] LiteralInput(description='Allows to create addition..., name='Margin', serializer='json', value=(5, 10))
            [11] DictInput(description='Dictionary of C..., name='Styles', serializer='json', type=<class 'dict'>)
            [12] ListInput(description='List of arbitrary t..., name='Tags', serializer='json', type=<class 'list'>)
            [13] IntInput(description='The width of the componen..., name='Width', start=0)
            [14] Select(description='Describes how the compone..., name='Width policy', options=OrderedDict([('auto', ...]), value='auto')
            [15] Select(description='Describes how the compone..., name='Height policy', options=OrderedDict([('auto', ...]), value='auto')
            [16] Select(description='How the component should ..., name='Sizing mode', options=OrderedDict([('fixed', ...]), value='stretch_width')
            [17] Checkbox(name='Visible', value=True)
    [1] Vizzu(dict, config={'geometry': 'rectangle', ...}, duration=400, height=400, sizing_mode='stretch_width', tooltip=True)